In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import math
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path
from typing import Any
from tqdm import tqdm
import warnings

In [ ]:
class InputEmbeddings(nn.Module):
    def __init__(self, d_model:int, vocab_size:int):
      super().__init__()
      self.d_model = d_model
      self.vocab_size = vocab_size
      self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
      return self.embedding(x) * math.sqrt(self.d_model)


In [ ]:
class PositionalEmbeddings(nn.Module):
  def __init__(self, d_model:int, max_seq_length:int,dropout:float):
    super().__init__()
    self.d_model = d_model
    self.max_seq_length = max_seq_length
    self.dropout = nn.Dropout(dropout)
    pe = torch.zeros(max_seq_length, d_model)
    pos = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(pos * div_term)
    pe[:, 1::2] = torch.cos(pos * div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer('pe', pe)

  def forward(self, x):
    # Explicitly expand positional encoding to match the input tensor's shape
    x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
    return self.dropout(x)

In [ ]:
class MultiheadattentionBlock(nn.Module):
  def __init__(self, d_model:int, h:int,dropout:float) -> None:
    super().__init__()
    self.d_model = d_model
    self.h = h
    assert d_model % h == 0, "d_model is not divisible by h"
    self.d_k = d_model // h
    self.dropout = nn.Dropout(dropout)
    self.W_q = nn.Linear(d_model, d_model)
    self.W_k = nn.Linear(d_model, d_model)
    self.W_v = nn.Linear(d_model, d_model)
    self.W_o = nn.Linear(d_model, d_model)

  @staticmethod
  def attention(query, key, value, mask, dropout : nn.Dropout):
    d_k = query.size(-1)
    attention_scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
      attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
    attention_scores = attention_scores.softmax(dim=-1)
    if dropout is not None:
      attention_scores = dropout(attention_scores)
    return torch.matmul(attention_scores, value), attention_scores

  def forward(self, q, k, v, mask=None):
    query = self.W_q(q)
    key = self.W_k(k)
    value = self.W_v(v)

    query = query.view(query.shape[0],query.shape[1], self.h, self.d_k).transpose(1, 2)
    key = key.view(key.shape[0],key.shape[1], self.h, self.d_k).transpose(1, 2)
    value = value.view(value.shape[0],value.shape[1], self.h, self.d_k).transpose(1, 2)
    x, self.attention_scores = MultiheadattentionBlock.attention(query, key, value, mask, self.dropout)
    x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
    return self.W_o(x)


In [ ]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model:int, d_ff:int, dropout:float) -> None:
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model)

  def forward(self,x):
    return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

In [ ]:
class NormalisationBlock(nn.Module):
  def __init__(self, eps:float=1e-6) -> None:
    super().__init__()
    self.eps = eps
    self.alpha = nn.Parameter(torch.ones(1))
    self.bias = nn.Parameter(torch.zeros(1))

  def forward(self,x):
    mean = x.mean(dim= -1, keepdim=True)
    std = x.std(dim =-1, keepdim=True)
    return self.alpha * (x - mean) / (std + self.eps) + self.bias


In [ ]:
class residual_connection(nn.Module):
  def __init__(self, dropout:float) -> None:
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = NormalisationBlock()

  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderBlock(nn.Module):
  def __init__(self,self_attention_block:MultiheadattentionBlock, feed_forward_block:FeedForwardBlock, dropout:float) -> None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feed_forward_block = feed_forward_block
    self.dropout = dropout
    self.residual_connections = nn.ModuleList([residual_connection(dropout) for _ in range(2)])
  def forward(self, x, src_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
    x = self.residual_connections[1](x, self.feed_forward_block)
    return x

In [ ]:
class Encoder(nn.Module):
  def __init__(self, layers:nn.ModuleList) -> None:
    super().__init__()
    self.layers = layers

  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
    return x


In [ ]:
class DecoderBlock(nn.Module):
  def __init__(self, self_attention_block:MultiheadattentionBlock, cross_attention_block:MultiheadattentionBlock, feed_forward_block:FeedForwardBlock, dropout:float) -> None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward_block = feed_forward_block
    self.dropout = dropout
    self.residual_connection = nn.ModuleList([residual_connection(dropout) for _ in range(3)])

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    x = self.residual_connection[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
    x = self.residual_connection[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
    x = self.residual_connection[2](x, self.feed_forward_block)
    return x

In [ ]:
class Decoder(nn.Module):
  def __init__(self, layers:nn.ModuleList) -> None:
    super().__init__()
    self.layers = layers
    self.norm = NormalisationBlock()
  def forward(self, x, encoder_output, src_mask, tgt_mask):
    for layer in self.layers:
      x = layer(x, encoder_output, src_mask, tgt_mask)
    return self.norm(x)

In [ ]:
class Projectionlayer(nn.Module):
  def __init__(self,d_model:int, vocab_size:int) -> None:
    super().__init__()
    self.proj = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    return torch.log_softmax(self.proj(x), dim=-1)

In [ ]:
class Transformer(nn.Module):
  def __init__(self, encoder:Encoder, decoder:Decoder, src_embed:InputEmbeddings, tgt_embed:InputEmbeddings, src_pos:PositionalEmbeddings, tgt_pos:PositionalEmbeddings, projection_layer:Projectionlayer) -> None:
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

  def encode(self, src, src_mask):
    src = self.src_embed(src)
    src = self.src_pos(src)
    return self.encoder(src,src_mask)

  def decode(self, encoder_output, src_mask, tgt, tgt_mask):
    tgt = self.tgt_embed(tgt)
    tgt = self.tgt_pos(tgt)
    return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

  def project(self,x):
    return self.projection_layer(x)

In [ ]:
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8, dropout: float = 0.1, d_ff: int = 2048) -> Transformer:

    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)
    src_pos = PositionalEmbeddings(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEmbeddings(d_model, tgt_seq_len, dropout)

    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiheadattentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)


        encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)


    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiheadattentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiheadattentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)


        decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))


    projection_layer = Projectionlayer(d_model, tgt_vocab_size)


    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)


    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

In [ ]:
def get_all_sentences(ds, lang):
    for pair in ds:
        yield pair['translation'][lang]

In [ ]:
def build_tokenizer(config, ds, lang):

    # Crating a file path for the tokenizer
    tokenizer_path = Path(config['tokenizer_file'].format(lang))

    # Checking if Tokenizer already exists
    if not Path.exists(tokenizer_path):

        # If it doesn't exist, we create a new one
        tokenizer = Tokenizer(WordLevel(unk_token = '[UNK]')) # Initializing a new world-level tokenizer
        tokenizer.pre_tokenizer = Whitespace() # We will split the text into tokens based on whitespace

        # Creating a trainer for the new tokenizer
        trainer = WordLevelTrainer(special_tokens = ["[UNK]", "[PAD]",
                                                     "[SOS]", "[EOS]"], min_frequency = 2) # Defining Word Level strategy and special tokens

        # Training new tokenizer on sentences from the dataset and language specified
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer = trainer)
        tokenizer.save(str(tokenizer_path)) # Saving trained tokenizer to the file path specified at the beginning of the function
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path)) # If the tokenizer already exist, we load it
    return tokenizer # Returns the loaded tokenizer or the trained tokenizer

In [ ]:
def get_ds(config):

    # Loading the train portion of the OpusBooks dataset.
    # The Language pairs will be defined in the 'config' dictionary we will build later
    ds_raw = load_dataset('opus_books', f'{config["lang_src"]}-{config["lang_tgt"]}', split = 'train')

    # Building or loading tokenizer for both the source and target languages
    tokenizer_src = build_tokenizer(config, ds_raw, config['lang_src'])
    tokenizer_tgt = build_tokenizer(config, ds_raw, config['lang_tgt'])

    # Splitting the dataset for training and validation
    train_ds_size = int(0.9 * len(ds_raw)) # 90% for training
    val_ds_size = len(ds_raw) - train_ds_size # 10% for validation
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size]) # Randomly splitting the dataset

    # Processing data with the BilingualDataset class, which we will define below
    train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

    # Iterating over the entire dataset and printing the maximum length found in the sentences of both the source and target languages
    max_len_src = 0
    max_len_tgt = 0
    for pair in ds_raw:
        src_ids = tokenizer_src.encode(pair['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_src.encode(pair['translation'][config['lang_tgt']]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

    print(f'Max length of source sentence: {max_len_src}')
    print(f'Max length of target sentence: {max_len_tgt}')

    # Creating dataloaders for the training and validadion sets
    # Dataloaders are used to iterate over the dataset in batches during training and validation
    train_dataloader = DataLoader(train_ds, batch_size = config['batch_size'], shuffle = True) # Batch size will be defined in the config dictionary
    val_dataloader = DataLoader(val_ds, batch_size = 1, shuffle = True)

    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

In [ ]:
def casual_mask(size):
  # Creating a square matrix of dimensions 'size x size' filled with ones
  mask = torch.triu(torch.ones(1, size, size), diagonal = 1).type(torch.int)
  return mask == 0

In [ ]:
class BilingualDataset(Dataset):

    # This takes in the dataset contaning sentence pairs, the tokenizers for target and source languages, and the strings of source and target languages
    # 'seq_len' defines the sequence length for both languages
    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len) -> None:
        super().__init__()

        self.seq_len = seq_len
        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

        # Defining special tokens by using the target language tokenizer
        self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64)


    # Total number of instances in the dataset (some pairs are larger than others)
    def __len__(self):
        return len(self.ds)

    # Using the index to retrive source and target texts
    def __getitem__(self, index: Any) -> Any:
        src_target_pair = self.ds[index]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]

        # Tokenizing source and target texts
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

        # Computing how many padding tokens need to be added to the tokenized texts
        # Source tokens
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2 # Subtracting the two '[EOS]' and '[SOS]' special tokens
        # Target tokens
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1 # Subtracting the '[SOS]' special token

        # If the texts exceed the 'seq_len' allowed, it will raise an error. This means that one of the sentences in the pair is too long to be processed
        # given the current sequence length limit (this will be defined in the config dictionary below)
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError('Sentence is too long')

        # Building the encoder input tensor by combining several elements
        encoder_input = torch.cat(
            [
            self.sos_token, # inserting the '[SOS]' token
            torch.tensor(enc_input_tokens, dtype = torch.int64), # Inserting the tokenized source text
            self.eos_token, # Inserting the '[EOS]' token
            torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype = torch.int64) # Addind padding tokens
            ]
        )

        # Building the decoder input tensor by combining several elements
        decoder_input = torch.cat(
            [
                self.sos_token, # inserting the '[SOS]' token
                torch.tensor(dec_input_tokens, dtype = torch.int64), # Inserting the tokenized target text
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype = torch.int64) # Addind padding tokens
            ]

        )

        # Creating a label tensor, the expected output for training the model
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype = torch.int64), # Inserting the tokenized target text
                self.eos_token, # Inserting the '[EOS]' token
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype = torch.int64) # Adding padding tokens

            ]
        )

        # Ensuring that the length of each tensor above is equal to the defined 'seq_len'
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            'encoder_input': encoder_input,
            'decoder_input': decoder_input,
            'encoder_mask': (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
            'decoder_mask': (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & casual_mask(decoder_input.size(0)),
            'label': label,
            'src_text': src_text,
            'tgt_text': tgt_text
        }

In [ ]:
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    # Retrieving the indices from the start and end of sequences of the target tokens
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Computing the output of the encoder for the source sequence
    encoder_output = model.encode(source, source_mask)
    # Initializing the decoder input with the Start of Sentence token
    decoder_input = torch.empty(1,1).fill_(sos_idx).type_as(source).to(device)

    # Looping until the 'max_len', maximum length, is reached
    while True:
        if decoder_input.size(1) == max_len:
            break

        # Building a mask for the decoder input
        decoder_mask = casual_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # Calculating the output of the decoder
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # Applying the projection layer to get the probabilities for the next token
        prob = model.project(out[:, -1])

        # Selecting token with the highest probability
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, torch.empty(1,1). type_as(source).fill_(next_word.item()).to(device)], dim=1)

        # If the next token is an End of Sentence token, we finish the loop
        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)

In [ ]:
def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_state, writer, num_examples=2):
    model.eval() # Setting model to evaluation mode
    count = 0 # Initializing counter to keep track of how many examples have been processed

    console_width = 80 # Fixed witdh for printed messages

    # Creating evaluation loop
    with torch.no_grad(): # Ensuring that no gradients are computed during this process
        for batch in validation_ds:
            count += 1
            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)

            # Ensuring that the batch_size of the validation set is 1
            assert encoder_input.size(0) ==  1, 'Batch size must be 1 for validation.'

            # Applying the 'greedy_decode' function to get the model's output for the source text of the input batch
            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            # Retrieving source and target texts from the batch
            source_text = batch['src_text'][0]
            target_text = batch['tgt_text'][0] # True translation
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy()) # Decoded, human-readable model output

            # Printing results
            print_msg('-'*console_width)
            print_msg(f'SOURCE: {source_text}')
            print_msg(f'TARGET: {target_text}')
            print_msg(f'PREDICTED: {model_out_text}')

            # After two examples, we break the loop
            if count == num_examples:
                break

In [ ]:
def get_model(config, vocab_src_len, vocab_tgt_len):

    # Loading model using the 'build_transformer' function.
    # We will use the lengths of the source language and target language vocabularies, the 'seq_len', and the dimensionality of the embeddings
    model = build_transformer(vocab_src_len, vocab_tgt_len, config['seq_len'], config['seq_len'], config['d_model'])
    return model


In [ ]:
def get_config():
    return{
        'batch_size': 8,
        'num_epochs': 20,
        'lr': 10**-4,
        'seq_len': 350,
        'd_model': 512,
        'lang_src': 'en',
        'lang_tgt': 'it',
        'model_folder': '/content/drive/MyDrive/transformer_weights',  # Changed to Google Drive path
        'model_basename': 'tmodel_',
        'preload': 'latest',
        'tokenizer_file': '/content/drive/MyDrive/tokenizer_{0}.json',  # Also save tokenizers to Drive
        'experiment_name': '/content/drive/MyDrive/runs/tmodel'  # Save tensorboard logs too
    }


# Function to construct the path for saving and retrieving model weights
def get_weights_file_path(config, epoch: str):
    model_folder = config['model_folder'] # Extracting model folder from the config
    model_basename = config['model_basename'] # Extracting the base name for model files
    model_filename = f"{model_basename}{epoch}.pt" # Building filename
    return str(Path('.')/ model_folder/ model_filename)

In [ ]:
def get_latest_weights_file_path(config):
    """Find the latest checkpoint file"""
    model_folder = config['model_folder']
    model_basename = config['model_basename']

    # Get all weight files
    weights_files = list(Path(model_folder).glob(f"{model_basename}*.pt"))

    if len(weights_files) == 0:
        return None

    # Sort by epoch number and return the latest
    weights_files.sort()
    return str(weights_files[-1])

In [ ]:
def train_model(config):
    # Setting up device to run on GPU to train faster
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device {device}")

    # Creating model directory to store weights
    Path(config['model_folder']).mkdir(parents=True, exist_ok=True)

    # Retrieving dataloaders and tokenizers for source and target languages using the 'get_ds' function
    train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

    # Initializing model on the GPU using the 'get_model' function
    model = get_model(config,tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

    # Tensorboard
    writer = SummaryWriter(config['experiment_name'])

    # Setting up the Adam optimizer with the specified learning rate from the '
    # config' dictionary plus an epsilon value
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps = 1e-9)

    # Initializing epoch and global step variables
    initial_epoch = 0
    global_step = 0

    # Checking if there is a pre-trained model to load
    # If true, loads it
    # Checking if there is a pre-trained model to load
# If true, loads it
    # Checking if there is a pre-trained model to load
    if config['preload'] == 'latest':
        model_filename = get_latest_weights_file_path(config)
        if model_filename:
            print(f'Preloading model {model_filename}')
            state = torch.load(model_filename)
            model.load_state_dict(state['model_state_dict'])
            initial_epoch = state['epoch'] + 1
            optimizer.load_state_dict(state['optimizer_state_dict'])
            global_step = state['global_step']
            print(f'Resuming from epoch {initial_epoch}')
        else:
            print('No checkpoint found, starting from scratch')
    elif config['preload']:
        model_filename = get_weights_file_path(config, config['preload'])
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        model.load_state_dict(state['model_state_dict'])
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']

    # Initializing CrossEntropyLoss function for training
    # We ignore padding tokens when computing loss, as they are not relevant for the learning process
    # We also apply label_smoothing to prevent overfitting
    loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer_src.token_to_id('[PAD]'), label_smoothing = 0.1).to(device)

    # Initializing training loop

    # Iterating over each epoch from the 'initial_epoch' variable up to
    # the number of epochs informed in the config
    for epoch in range(initial_epoch, config['num_epochs']):

        # Initializing an iterator over the training dataloader
        # We also use tqdm to display a progress bar
        batch_iterator = tqdm(train_dataloader, desc = f'Processing epoch {epoch:02d}')

        # For each batch...
        for batch in batch_iterator:
            model.train() # Train the model

            # Loading input data and masks onto the GPU
            encoder_input = batch['encoder_input'].to(device)
            decoder_input = batch['decoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)
            decoder_mask = batch['decoder_mask'].to(device)

            # Running tensors through the Transformer
            encoder_output = model.encode(encoder_input, encoder_mask)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
            proj_output = model.project(decoder_output)

            # Loading the target labels onto the GPU
            label = batch['label'].to(device)

            # Computing loss between model's output and true labels
            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))

            # Updating progress bar
            batch_iterator.set_postfix({f"loss": f"{loss.item():6.3f}"})

            writer.add_scalar('train loss', loss.item(), global_step)
            writer.flush()

            # Performing backpropagation
            loss.backward()

            # Updating parameters based on the gradients
            optimizer.step()

            # Clearing the gradients to prepare for the next batch
            optimizer.zero_grad()

            global_step += 1 # Updating global step count

        # We run the 'run_validation' function at the end of each epoch
        # to evaluate model performance
        run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step, writer)

        # Saving model
        model_filename = get_weights_file_path(config, f'{epoch:02d}')
        # Writting current model state to the 'model_filename'
        torch.save({
            'epoch': epoch, # Current epoch
            'model_state_dict': model.state_dict(),# Current model state
            'optimizer_state_dict': optimizer.state_dict(), # Current optimizer state
            'global_step': global_step # Current global step
        }, model_filename)

In [ ]:
if __name__ == '__main__':
    warnings.filterwarnings('ignore') # Filtering warnings
    config = get_config() # Retrieving config settings
    train_model(config) # Training model with the config arguments

Using device cuda


README.md: 0.00B [00:00, ?B/s]

en-it/train-00000-of-00001.parquet:   0%|          | 0.00/5.73M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of source sentence: 309
Max length of target sentence: 274
Preloading model /content/drive/MyDrive/transformer_weights/tmodel_17.pt
Resuming from epoch 18


Processing epoch 18: 100%|██████████| 3638/3638 [25:33<00:00,  2.37it/s, loss=2.163]


--------------------------------------------------------------------------------
SOURCE: Let's go on a bit further, and see if there isn't one with honeysuckle over it."
TARGET: Andiamo più oltre a cercare un albergo ornato di caprifoglio.
PREDICTED: Andiamo a un albergo più , e vedere se non c ’ è un albergo con un albergo .
--------------------------------------------------------------------------------
SOURCE: TOWARD TEN O'CLOCK the old Prince, Koznyshev, and Oblonsky were with Levin, and having talked about the young mother they had begun discussing other matters.
TARGET: Dopo le nove il vecchio principe, Sergej Ivanovic e Stepan Arkad’ic erano dai Levin e, dopo aver parlato della puerpera, si misero a discorrere anche di argomenti estranei.
PREDICTED: Dopo le nove il vecchio principe , Sergej Ivanovic e Stepan Arkad ’ ic erano con Levin e , dopo aver parlato della madre , si misero a parlare della .


Processing epoch 19: 100%|██████████| 3638/3638 [25:40<00:00,  2.36it/s, loss=1.918]


--------------------------------------------------------------------------------
SOURCE: This was her thoroughbred quality – the kind of blood that tells, as they say in English.
TARGET: Ma possedeva al massimo una qualità che faceva dimenticare tutti i suoi difetti; aveva il sangue, sangue “che si fa sentire”, come dicono gli inglesi.
PREDICTED: Ma possedeva al sangue , qualità che faceva possedeva in inglese .
--------------------------------------------------------------------------------
SOURCE: Mama is coming and Grandpapa and Sergius Ivanich, and some one else!' they cried, clambering into the trap.
TARGET: Stanno venendo la mamma e il nonno, e Sergej Ivanyc, e ancora qualcuno — dicevano, arrampicandosi sul calesse.
PREDICTED: Mamma sarà venendo la mamma e Sergej Ivanovic , ed altri ! — dicevano , in calesse accanto al calesse .


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.0 MB/s eta 0:00:00


In [ ]:
def evaluate_model(config, checkpoint_epoch='latest'):
    """Evaluate a trained model and calculate BLEU score"""
    import sacrebleu

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device {device}")

    # Load data and tokenizers
    _, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

    # Load model
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

    # Load checkpoint
    if checkpoint_epoch == 'latest':
        model_filename = get_latest_weights_file_path(config)
    else:
        model_filename = get_weights_file_path(config, checkpoint_epoch)

    print(f'Loading model from {model_filename}')
    state = torch.load(model_filename, map_location=device)
    model.load_state_dict(state['model_state_dict'])
    print(f'Model loaded from epoch {state["epoch"]}')

    # Evaluate
    model.eval()
    predictions = []
    references = []

    print("Generating translations...")
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Evaluating"):
            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, config['seq_len'], device)

            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())
            target_text = batch['tgt_text'][0]

            predictions.append(model_out_text)
            references.append([target_text])

    # Calculate BLEU
    bleu = sacrebleu.corpus_bleu(predictions, references)
    print(f'\n{"="*50}')
    print(f'BLEU Score: {bleu.score:.2f}')
    print(f'{"="*50}')

    return bleu.score, predictions, references

# Run evaluation
config = get_config()
bleu_score, predictions, references = evaluate_model(config, checkpoint_epoch='latest')

# Optional: Show some examples
print("\nSample Translations:")
for i in range(min(5, len(predictions))):
    print(f'\nExample {i+1}:')
    print(f'Reference:  {references[i][0]}')
    print(f'Predicted:  {predictions[i]}')

Using device cuda
Max length of source sentence: 309
Max length of target sentence: 274
Loading model from /content/drive/MyDrive/transformer_weights/tmodel_19.pt
Model loaded from epoch 19
Generating translations...


Evaluating: 100%|██████████| 3234/3234 [15:19<00:00,  3.52it/s]



BLEU Score: 73.23

Sample Translations:

Example 1:
Reference:  Il mese assegnato dal signor Rochester era trascorso e le ore che mi dividevano dal matrimonio erano contate e non era possibile di ritardare la cerimonia, perché tutto era pronto.
Predicted:  Il mese assegnato dal mese era trascorso e le ore che non era possibile di ritardare la cerimonia , perché tutto era pronto e non era pronto .

Example 2:
Reference:  — Ma è a cento miglia di qui.
Predicted:  — Ma è a cento miglia di qui .

Example 3:
Reference:  Potrebbe essere stato un poco urtato per l'irregolarità dei miei tratti, essendo i suoi così armoniosi.
Predicted:  Potrebbe essere stato un poco urtato per l ' irregolarità dei miei tratti essendo i suoi così armoniosi .

Example 4:
Reference:  Doveva quindi calmarsi e uscire da quello stato di angoscia. Il pensiero di un’azione immediata collegata col figlio, il fatto di dover partire con lui per un luogo lontano, le dette questa tranquillità.
Predicted:  Doveva quindi ca